In [9]:
# Getting the libraries 
import os
import re
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [10]:
# Function to extract time from filename
def extract_time(filename):
    match = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}%3A\d{2}%3A\d{2}', filename)
    return match.group() if match else 'unknown'

In [11]:
# Specify the directory containing the WRF output files
wrf_dir = r'D:\PhD_works\Simulations_output\150424\blowingsnow_GFS_MYJ\with\2020021100\data'  # replace with your directory

# Get a list of all NetCDF files in the wrf_dir
files = [os.path.join(wrf_dir, f) for f in os.listdir(wrf_dir) if os.path.isfile(os.path.join(wrf_dir, f)) and f.startswith('wrfout')]

In [12]:
# Let's plot the wind now
for file in files:
    # Open the NetCDF file
    ds = nc.Dataset(file)

    # Extract data
    u10 = ds.variables['U10'][0, ::10, ::10]  # Select every 10th point
    v10 = ds.variables['V10'][0, ::10, ::10]  # Select every 10th point
    xlat = ds.variables['XLAT'][0, ::10, ::10]  # Select every 10th point
    xlong = ds.variables['XLONG'][0, ::10, ::10]  # Select every 10th point

    # Create a new figure
    fig = plt.figure(figsize=(10, 10))

    # Create a GeoAxes in the tile's projection
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Add features to the map
    ax.add_feature(cfeature.BORDERS, edgecolor = 'red')
    ax.add_feature(cfeature.STATES, edgecolor = 'black')
    ax.add_feature(cfeature.RIVERS, edgecolor = 'blue')
    ax.add_feature(cfeature.LAKES, edgecolor = 'blue')

    # Plot wind barbs
    ax.barbs(xlong, xlat, u10, v10, length=5)

    # Add gridlines and labels for lat/lon
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False

    # Extract date and time from filename
    filename, _ = os.path.splitext(os.path.basename(file))
    _, _, date, time = filename.split('_')
    
    # Keep only the date and hour
    date, hour = date, time.split('%3A')[0]
    datetime_str = f'{date} {hour}'
    
    # Set the title to include the variable name and datetime
    ax.set_title(f'10m Wind (m/s) at {datetime_str}Z')

    # Remove white spaces around the plot
    plt.tight_layout()

    # Save the figure in the script's directory
    plt.savefig(os.path.basename(file) + '_wind_plot.png', dpi = 300, bbox_inches = 'tight')

    # Close the figure to free up memory
    plt.close(fig)

print("Wind plots have been saved as PNG files.")

Wind plots have been saved as PNG files.
